In [43]:
from ase import Atoms
from ase.md.npt import NPT
from ase.md import VelocityVerlet
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
import ase.io as io
import numpy as np
from ase import units as ase_units
from mbpol_calculator import *
from ase.md import MDLogger

T = 298.15
dt = 0.5
ttime = 0.015
    
a = 15.646 

h2o = Atoms('128OHH',
           positions = np.genfromtxt('../128.csv',delimiter = ','),
           cell = [a, a, a],
           pbc = True)

h2o_shifted = reconnect_monomers(h2o)

h2o.calc = MbpolCalculator(h2o)

# Setting the initial T 100 K lower leads to faster convergence of initial oscillations
MaxwellBoltzmannDistribution(h2o, 200 * ase_units.kB)

#    while(abs(h2o.get_temperature() - 300) > 1):
#        MaxwellBoltzmannDistribution(h2o, 300 * ase_units.kB)
print('ttime= {} fs :: temperature = {}'.format(ttime,h2o.get_temperature()))


h2o.set_momenta(h2o.get_momenta() - np.mean(h2o.get_momenta(),axis =0))

traj = io.Trajectory('./test.traj', mode = 'a', atoms = h2o)


dyn = NPT(h2o, timestep = dt * ase_units.fs, 
          temperature =  T * ase_units.kB, externalstress = 0,
          ttime = ttime * ase_units.fs, pfactor = None,
                     trajectory=traj.write)
logfile = MDLogger(dyn, h2o, './test.log', header=False, stress=False, mode="a")

dyn.attach(logfile, interval = 1)

dyn.run(5) 
logfile.close()

Monomers were reconnected
ttime= 0.015 fs :: temperature = 214.6779731084554


In [44]:
! cat test.log

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.001          -37.649      -37.649        0.000     0.0
0.001          -36.458      -36.458        0.000     0.0
0.001          -37.648      -37.648        0.000     0.0
0.002          -36.453      -36.453        0.000     0.0
0.003          -37.648      -37.648        0.000     0.0
0.001          -37.616      -37.616        0.001     0.0
0.001          -36.450      -36.450        0.000     0.0
0.001          -37.615      -37.615        0.000     0.0
0.002          -36.445      -36.445        0.000     0.0
0.003          -37.615      -37.615        0.000     0.0
